In [1]:
"""
FULL CONVOLUTIONAL MODEL TEST
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os
import utils
import tensorflow as tf
from config import cfg
import conv_bodies
import modeling
import matplotlib.pyplot as plt


model, weights_file, start_iter, checkpoints, output_dir = modeling.create_model()
model.set_dataset('train_data/', 'train_labels/')

if model.train:
    model.tensor, model.labels = utils.stack_batch_into_tensor(model.dataset[0], train=model.train)
else:
    model.tensor = utils.stack_batch_into_tensor(model.dataset[0], train=model.train)
    
init = (tf.global_variables_initializer(), 
        tf.local_variables_initializer(), 
        model.dataset[1].make_initializer(model.dataset[0]))

with tf.Session() as sess:
    sess.run(init)
    original_shape = sess.run(model.tensor).shape
    final_shape = original_shape[:3] + (150,)
    print('Starting network', model.tensor.shape)
    with tf.variable_scope('conv1', reuse=tf.AUTO_REUSE):
        model.Conv(3, 3, 64, padding='SAME', strides=[1,1,1,1])
        model.Relu()
    print('Layer 1 done', model.tensor.shape)
    with tf.variable_scope('conv2', reuse=tf.AUTO_REUSE):
        model.Conv(3, 64, 64, padding='SAME', strides=[1,1,1,1])
        model.Relu()
        # Strides[0] must be 1, else
        # UnimplementError Pooling is not yet supported on the batch dimension
        model.MaxPool(ksize=[1,1,2,1], strides=[1,2,2,1], padding='SAME')
    print('Layer 2 done', model.tensor.shape)
    with tf.variable_scope('conv3', reuse=tf.AUTO_REUSE):
        model.Conv(3, 64, 128, padding='SAME', strides=[1,1,1,1])
        model.Relu()
    print('Layer 3 done', model.tensor.shape)
    with tf.variable_scope('conv4', reuse=tf.AUTO_REUSE):
        model.Conv(3, 128, 128, padding='SAME', strides=[1,1,1,1])
        model.Relu()
        model.MaxPool(ksize=[1,1,2,1], strides=[1,2,2,1], padding='SAME')
    print('Layer 5 done', model.tensor.shape)
    with tf.variable_scope('conv5', reuse=tf.AUTO_REUSE):
        model.Conv(3, 128, 256, padding='SAME', strides=[1,1,1,1])
        model.Relu()
    print('Layer 5 done', model.tensor.shape)
    with tf.variable_scope('conv6', reuse=tf.AUTO_REUSE):
        model.Conv(3, 256, 256, padding='SAME', strides=[1,1,1,1])
        model.Relu()
    print('Layer 6 done', model.tensor.shape)
    with tf.variable_scope('conv7', reuse=tf.AUTO_REUSE):
        model.Conv(3, 256, 256, padding='SAME', strides=[1,1,1,1])
        model.Relu()
        model.MaxPool(ksize=[1,1,2,1], strides=[1,2,2,1], padding='SAME')
        pool3 = model.tensor
    print('Layer 7 done', model.tensor.shape)
    # Get scores for pool3
    with tf.variable_scope('conv16', reuse=tf.AUTO_REUSE):
        filter_shape = [1, 1, 256, 150]
        weights = tf.get_variable('weights', shape=filter_shape).initialized_value()
        pool3 = tf.nn.conv2d(pool3, weights, padding='SAME', strides=[1,1,1,1])
    print('pool3 scores', pool3.shape)
    with tf.variable_scope('conv8', reuse=tf.AUTO_REUSE):
        model.Conv(3, 256, 512, padding='SAME', strides=[1,1,1,1])
        model.Relu()
    print('Layer 8 done', model.tensor.shape)
    with tf.variable_scope('conv9', reuse=tf.AUTO_REUSE):
        model.Conv(3, 512, 512, padding='SAME', strides=[1,1,1,1])
        model.Relu()
    print('Layer 9 done', model.tensor.shape)
    with tf.variable_scope('conv10', reuse=tf.AUTO_REUSE):
        model.Conv(3, 512, 512, padding='SAME', strides=[1,1,1,1])
        model.Relu()
        model.MaxPool(ksize=[1,1,2,1], strides=[1,2,2,1], padding='SAME')
        pool4 = model.tensor
    print('Layer 10 done', model.tensor.shape)
    # Get scores for pool4
    with tf.variable_scope('conv17', reuse=tf.AUTO_REUSE):
        filter_shape = [1, 1, 512, 150]
        weights = tf.get_variable('weights', shape=filter_shape).initialized_value()
        pool4 = tf.nn.conv2d(pool4, weights, padding='SAME', strides=[1,1,1,1])
    print('pool4 scores', pool4.shape)
    with tf.variable_scope('conv11', reuse=tf.AUTO_REUSE):
        model.Conv(3, 512, 512, padding='SAME', strides=[1,1,1,1])
        model.Relu()
    print('Layer 11 done', model.tensor.shape)
    with tf.variable_scope('conv12', reuse=tf.AUTO_REUSE):
        model.Conv(3, 512, 512, padding='SAME', strides=[1,1,1,1])
        model.Relu()
    print('Layer 12 done', model.tensor.shape)
    with tf.variable_scope('conv13', reuse=tf.AUTO_REUSE):
        model.Conv(3, 512, 512, padding='SAME', strides=[1,1,1,1])
        model.Relu()
        model.MaxPool(ksize=[1,1,2,1], strides=[1,2,2,1], padding='SAME')
    print('Layer 13 done', model.tensor.shape)
    with tf.variable_scope('conv14', reuse=tf.AUTO_REUSE):
        model.Conv(3, 512, 4096, padding='SAME', strides=[1,1,1,1])
        model.Relu()
    print('Layer 14 done', model.tensor.shape)
    with tf.variable_scope('conv15', reuse=tf.AUTO_REUSE):
        model.Conv(3, 4096, 4096, padding='SAME', strides=[1,1,1,1])
        model.Relu()
        conv7 = model.tensor
    print('Layer 15 done', model.tensor.shape)
    # Get scores for conv7
    with tf.variable_scope('conv18', reuse=tf.AUTO_REUSE):
        filter_shape = [1, 1, 4096, model.num_classes]
        weights = tf.get_variable('weights', shape=filter_shape).initialized_value()
        conv7 = tf.nn.conv2d(conv7, weights, padding='SAME', strides=[1,1,1,1])
    print('conv7 scores', conv7.shape)
    
    # Upsample coarse outputs
    with tf.variable_scope('deconv1', reuse=tf.AUTO_REUSE):
        filter_shape = [3, 3, model.num_classes, model.num_classes]
        weights = tf.get_variable('weights', shape=filter_shape).initialized_value()
        pool4_2x = tf.nn.conv2d_transpose(pool4, weights, pool3.shape, padding='SAME', strides=[1,2,2,1])
    print('pool4_2x', pool4_2x.shape)
        
    with tf.variable_scope('deconv2', reuse=tf.AUTO_REUSE):
        filter_shape = [3, 3, model.num_classes, model.num_classes]
        weights = tf.get_variable('weights', shape=filter_shape).initialized_value()
        conv7_4x = tf.nn.conv2d_transpose(conv7, weights, pool3.shape, padding='SAME', strides=[1,4,4,1])
    print('conv7_4x', conv7_4x.shape)

    
    # Add Tensors
    model.tensor = tf.add(pool3, pool4_2x)
    model.tensor = tf.add(model.tensor, conv7_4x)
    
    # Final Upsampling
    with tf.variable_scope('deconv3', reuse=tf.AUTO_REUSE):
        filter_shape = [3, 3, model.num_classes, model.num_classes]
        weights = tf.get_variable('weights', shape=filter_shape).initialized_value()
        model.tensor = tf.nn.conv2d_transpose(model.tensor, weights, final_shape, padding='SAME', strides=[1,8,8,1])
    print('Final layer done', model.tensor.shape)
    print('Finished network')
    
    


Starting network (2, 224, 224, 3)
Layer 1 done (2, 224, 224, 64)
Layer 2 done (2, 112, 112, 64)
Layer 3 done (2, 112, 112, 128)
Layer 5 done (2, 56, 56, 128)
Layer 5 done (2, 56, 56, 256)
Layer 6 done (2, 56, 56, 256)
Layer 7 done (2, 28, 28, 256)
pool3 scores (2, 28, 28, 150)
Layer 8 done (2, 28, 28, 512)
Layer 9 done (2, 28, 28, 512)
Layer 10 done (2, 14, 14, 512)
pool4 scores (2, 14, 14, 150)
Layer 11 done (2, 14, 14, 512)
Layer 12 done (2, 14, 14, 512)
Layer 13 done (2, 7, 7, 512)
Layer 14 done (2, 7, 7, 4096)
Layer 15 done (2, 7, 7, 4096)
conv7 scores (2, 7, 7, 150)
pool4_2x (2, 28, 28, 150)
conv7_4x (2, 28, 28, 150)
Final layer done (2, 224, 224, 150)
Finished network
